<h1> <center> CS329e Darwn Project </center> </h1> 

<h2> <center> Name of Project </center> </h2>

<center> Explanation of what the project is and it's importance <center>

Links to use markdown and command mode to make this more readible (https://gist.github.com/kidpixo/f4318f8c8143adee5b40) (https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet)

Example: Press esc (enables command mode) then press b to make a cell below, press esc then m to allow it to be text and not code. Use shift+enter in order to move to turn it into nice text and move to next cell

## Import libraries

In [1]:
# Import necessary libraries
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image
from time import sleep
import os
import numpy as np
from sklearn.metrics import classification_report

from amb_sdk.sdk import DarwinSdk

## Setup

In [2]:
# Login
ds = DarwinSdk()
ds.set_url('https://amb-demo-api.sparkcognition.com/v1/')
status, msg = ds.auth_login_user('mc.mitchell.67@gmail.com', 'kPRk5SWBGv')

if not status:
    print(msg)

**Data Path**

In [3]:
path = './datasets'
dataset_name = 'mergedData.csv'
df = pd.read_csv(os.path.join(path,dataset_name))
df.head()

,Unnamed: 0,Geo_FIPS,percSOV,OBJECTID,GEOID10,TRFIPS,CFIPS,SFIPS,CSA,CSA_Name,...,D5cr,D5cri,D5ce,D5cei,D5dr,D5dri,D5de,D5dei,Shape_Length,Shape_Area
0,0,10010201001,0.887006,427,10010201001,20100,1,1,388.0,"Montgomery-Alexander City, AL",...,0.002123,0.31,0.25,0.45,-99999.0,-99999.0,-99999.0,-99999.0,0.110807,0.000410
1,1,10010201002,0.894172,1668,10010201002,20100,1,1,388.0,"Montgomery-Alexander City, AL",...,0.001654,0.24,0.20,0.36,-99999.0,-99999.0,-99999.0,-99999.0,0.096808,0.000534
2,2,10010202001,0.854737,217,10010202001,20200,1,1,388.0,"Montgomery-Alexander City, AL",...,0.001931,0.29,0.24,0.43,-99999.0,-99999.0,-99999.0,-99999.0,0.062129,0.000197
3,3,10010202002,0.973392,216,10010202002,20200,1,1,388.0,"Montgomery-Alexander City, AL",...,0.002335,0.35,0.28,0.50,-99999.0,-99999.0,-99999.0,-99999.0,0.052277,0.000124
4,4,10010203001,0.912568,207,10010203001,20300,1,1,388.0,"Montgomery-Alexander City, AL",...,0.002256,0.33,0.28,0.50,-99999.0,-99999.0,-99999.0,-99999.0,0.089441,0.000372


## Combine Data

**Combine data from the Smart Locations Database and Census Tract Data. Both datasets are for the entire U.S.**

Both datasets have a GEOID that is used to represent a geographic location

Some of the most common administrative/legal and statistical geographic entities with unique GEOIDs include states, counties, congressional districts, core based statistical areas (metropolitan and micropolitan areas), census tracts, block groups and census blocks. <br>

*Note: GEOIDs are numeric codes that uniquely identify all administrative/legal and statistical geographic areas for which the Census Bureau tabulates data.*

In [ ]:
# read data

In [ ]:
# combine data

In [ ]:
# export combined data as csv
# to datasets folder

## Data Upload and Clean

**Read dataset and view a file snippet**

After setting up the dataset path, the next step is to upload the dataset from your local device to the server. <br> In the cell below, you need to specify the dataset_name if you want to use your own data.

**Upload dataset to Darwin**

In [4]:
# Upload dataset
status, dataset = ds.upload_dataset(os.path.join(path, dataset_name))
if not status:
    print(dataset)

**Clean dataset**

In [5]:
# clean dataset
target = "percSOV"
status, job_id = ds.clean_data(dataset_name, target = target)

if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

{'status': 'Requested', 'starttime': '2019-04-02T16:03:12.492973', 'endtime': None, 'percent_complete': 0, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['mergedData.csv'], 'artifact_names': ['2c1e8a7b5f9743df98e30f8c01e540d1'], 'model_name': None, 'job_error': None}
{'status': 'Running', 'starttime': '2019-04-02T16:03:12.492973', 'endtime': None, 'percent_complete': 0, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['mergedData.csv'], 'artifact_names': ['2c1e8a7b5f9743df98e30f8c01e540d1'], 'model_name': None, 'job_error': ''}
{'status': 'Running', 'starttime': '2019-04-02T16:03:12.492973', 'endtime': None, 'percent_complete': 0, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['mergedData.csv'], 'artifact_names': ['2c1e8a7b5f9743df98e30f8c01e540d1'], 'model_name': None, 'job_error': ''}
{'status': 'Complete', 'starttime': '2019-04-02T16:03:12.492973', 'endtime': '2019-04-02T16:03:43.1

## Clean Dataset Model

We will now build a model that will learn the class labels in the target column.<br> In the default cancer dataset, the target column is "Diagnosis". <br> You will have to specify your own target name for your custom dataset. <br> You can also increase max_train_time for longer training.

In [ ]:
model = target + "_model0"
status, job_id = ds.create_model(dataset_names = dataset_name, \
                                 model_name =  model, \
                                 max_train_time = '00:02')
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)